# Demo
This notebook demonstrates cases of running our multi-player assistant.

## Set up
In order to run this demo, you need to set up .env file with open ai keys.

In [1]:
import sys

from loguru import logger

import sherpa_ai.config
from sherpa_ai.agents import (
    AgentPool,
    Critic,
    MLEngineer,
    Physicist,
    Planner,
    UserAgent,
)
from sherpa_ai.events import EventType
from sherpa_ai.memory import SharedMemory
from sherpa_ai.orchestrator import Orchestrator, OrchestratorConfig

2023-10-02 17:00:47.353 | INFO     | sherpa_ai.config:<module>:100 - Config: Slack environment variables are set
2023-10-02 17:00:47.454 | INFO     | sherpa_ai.config:<module>:106 - Config: OpenAI environment variables are set
2023-10-02 17:00:47.456 | INFO     | sherpa_ai.config:check_vectordb_setting:73 - Config: Pinecone environment variables are set. Using Pinecone database.


In [2]:
logger.remove()
logger.add(sys.stderr, level="INFO")   # change to DEBUG for more detail


user_description1 = """Sahar is skilled in astronomy and astrophysics. She has a PhD in those areas specializing in star formation.
Her expertise lies in developing and implementing models to study the formation mechanisms for galaxies and stars and applying computer vision algorithms to galaxy spectrum data.
She can answer questions about general astrophysics topics and also in depth technical details of modelling and galaxy and star formation questions and numerical methods related to these areas.
"""

user_description2 = """Amir is skilled in quantum physics, quantum information, quantum computing, quantum optics, quantum simulations, and atomic physics. He has a PhD in those areas specializing in experimental implementation of quantum computing gates.
His expertise lies in developing models to study the behavior of quantum optical systems, and building experimental setups to test out various hypothesis using data driven techniques.
He can answer questions about general quantum mechanics topics and also in depth technical details of modelling and implementing quantum optics experiments.
"""

## Test Planner/Critics

In [4]:
objective = "Write a proposal for estimating the maximum wind speed of a tropical cyclone using satellite imagery"
config = OrchestratorConfig(llm_name='gpt-3.5-turbo')  # gpt-3.5-turbo for cheaper api calls, gpt-4 for better performance but higher cost

orchestrator = Orchestrator(config=config)

shared_memeory = SharedMemory(
    objective=objective,
)
orchestrator.shared_memory = shared_memeory

physicist = Physicist(llm=orchestrator.llm, shared_memory=shared_memeory)

ml_engineer = MLEngineer(llm=orchestrator.llm, shared_memory=shared_memeory)

user_agent1 = UserAgent(
    name="Sahar",
    description=user_description1,
    shared_memory=shared_memeory,
)
user_agent2 = UserAgent(
    name="Amir", description=user_description2, shared_memory=shared_memeory
)

agent_pool = AgentPool()
agent_pool.add_agents([physicist, ml_engineer, user_agent1, user_agent2])

orchestrator.agent_pool = agent_pool

planner = Planner(
    agent_pool=agent_pool,
    shared_memory=shared_memeory,
    llm=orchestrator.llm,
    num_steps=5,
)

critic_agent = Critic(llm=orchestrator.llm, ratio=9, shared_memory=shared_memeory)
plan = orchestrator.plan(objective, planner, critic_agent)
print(plan)

2023-10-02 17:09:06.435 | INFO     | sherpa_ai.agents.planner:plan:70 - Plan: Step 1:
Agent: Sahar
Task: Gather satellite imagery data of the tropical cyclone.
Step 2:
Agent: ML Engineer
Task: Develop a machine learning model to analyze the satellite imagery data and extract relevant features.
Step 3:
Agent: Physicist
Task: Analyze the extracted features and determine the relationship between the features and the maximum wind speed of the tropical cyclone.
Step 4:
Agent: Amir
Task: Use the determined relationship to estimate the maximum wind speed of the tropical cyclone based on the analyzed satellite imagery data.
Step 5:
Agent: Physicist
Task: Validate and refine the estimated maximum wind speed using physics-based principles and statistical analysis.

2023-10-02 17:09:10.985 | INFO     | sherpa_ai.agents.critic:get_feedback:100 - i_score: 9, d_score: 9
2023-10-02 17:09:13.719 | INFO     | sherpa_ai.agents.critic:get_feedback:114 - feedback: 1. The plan is comprehensive and covers a

Step 1:
Agent: Sahar
Task: Gather satellite imagery data of the tropical cyclone.
Step 2:
Agent: ML Engineer
Task: Develop and train a machine learning model to analyze the satellite imagery data and extract relevant features.
Step 3:
Agent: Physicist
Task: Analyze the extracted features and identify potential correlations or patterns that may indicate the maximum wind speed of the tropical cyclone.
Step 4:
Agent: Amir
Task: Use the identified correlations or patterns to estimate the maximum wind speed of the tropical cyclone based on the analyzed satellite imagery data.
Step 5:
Agent: Physicist
Task: Validate and refine the estimated maximum wind speed using physics-based principles and statistical analysis techniques.



## Post-planning phase

Once we have a fixed plan, we can proceed to the assign steps to agents and complete each step

In [8]:
# plan_text = plan

plan_text = """Step 1:
Agent: ML Engineer
Task: Research and identify machine learning algorithms suitable for analyzing satellite imagery data to estimate wind speed of tropical cyclones. Specifically, focus on algorithms that have been successfully used for similar tasks in the past and have proven accuracy and reliability.
Step 2:
Agent: Physicist
Task: Collaborate with the physicist to understand the underlying physics principles and equations that govern the relationship between satellite imagery features and wind speed of tropical cyclones. Gather insights and guidance on how to incorporate physics-based models into the analysis. Discuss the potential benefits of integrating physics-based models with machine learning algorithms for more accurate estimation.
Step 3:
Agent: Sahar
Task: Consult with Sahar to understand the specific requirements and challenges related to using satellite imagery for estimating wind speed of tropical cyclones. Seek her expertise in computer vision algorithms and numerical methods for analyzing the satellite data. Discuss potential approaches and techniques that can be applied to enhance the accuracy of wind speed estimation.
Step 4:
Agent: ML Engineer
Task: Work with the ML Engineer to develop and train a machine learning model using the identified algorithms and the satellite imagery data. Collaborate on optimizing the model's performance and accuracy in estimating the maximum wind speed of tropical cyclones. Ensure that the chosen algorithms are appropriate for the task and can provide accurate results.
Step 5:
Agent: Sahar
Task: Consult with Sahar to explore the possibility of using computer vision algorithms and advanced image processing techniques to extract relevant features from the satellite imagery data. Collaborate on implementing these techniques to enhance the accuracy of wind speed estimation."""

plan = planner.planning.post_process(plan_text)
shared_memeory.add(EventType.planning, "Planner", str(plan))

assert len(plan.steps) > 0

assert user_agent1.shared_memory == physicist.shared_memory

orchestrator.execute(plan, planner)


2023-10-02 17:34:02.137 | INFO     | sherpa_ai.agents.base:run:53 - Action selected: ('ArxivSearch', {'query': '"machine learning algorithms for estimating wind speed of tropical cyclones using satellite imagery data"'})
2023-10-02 17:34:09.915 | INFO     | sherpa_ai.agents.base:run:71 - Action output: The paper titled "Estimating Wind Speed of Tropical Cyclones from Satellite Imagery Data using Machine Learning Algorithms" provides a comprehensive analysis of various machine learning algorithms suitable for estimating wind speed of tropical cyclones using satellite imagery data. The paper reviews and compares the performance of different algorithms, including Random Forest, Support Vector Regression, and Artificial Neural Networks, and concludes that Random Forest algorithm shows the highest accuracy and reliability for this task. The paper provides valuable insights and information for researchers and practitioners interested in using machine learning for analyzing satellite imagery 

@Sahar Please complete the following task: 
Consult with Sahar to understand the specific requirements and challenges related to using satellite imagery for estimating wind speed of tropical cyclones. Seek her expertise in computer vision algorithms and numerical methods for analyzing the satellite data. Discuss potential approaches and techniques that can be applied to enhance the accuracy of wind speed estimation.


2023-10-02 17:34:48.406 | INFO     | sherpa_ai.agents.base:run:53 - Action selected: ('Deliberation', {'task': '"Research and identify machine learning algorithms suitable for analyzing satellite imagery data to estimate wind speed of tropical cyclones"'})
2023-10-02 17:34:55.580 | INFO     | sherpa_ai.agents.base:run:71 - Action output: To research and identify machine learning algorithms suitable for analyzing satellite imagery data to estimate wind speed of tropical cyclones, the following steps can be taken:

1. Data Collection: Gather satellite imagery data that includes information on tropical cyclones and their corresponding wind speeds. This data should be diverse and representative of different cyclones and weather conditions.

2. Data Preprocessing: Prepare the data by cleaning and formatting it for analysis. This may involve removing noise or outliers, normalizing the data, and splitting it into training and testing sets.

3. Algorithm Selection: Evaluate different machine l

@Sahar Please complete the following task: 
Consult with Sahar to explore the possibility of using computer vision algorithms and advanced image processing techniques to extract relevant features from the satellite imagery data. Collaborate on implementing these techniques to enhance the accuracy of wind speed estimation.
